In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import GridSearchCV
import os
from sklearn.preprocessing import StandardScaler,StandardScaler, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from utils import load_config, tabulacion, apply_format, impute_missing_drop_columns
from preprocessing_cluster import preprocessing
import pickle
from sklearn.cluster import KMeans
from kmodes.kprototypes import KPrototypes
from sklearn.metrics import silhouette_score, pairwise_distances
import plotly.graph_objects as go
import xgboost as xgb
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
from clustering import clustering
import plotly.express as px

### Agrupación de animes con características similares

El análisis de clustering en los datos de anime permite identificar grupos naturales de animes según sus características. Esto puede ayudar a entender mejor al público objetivo y ayudar a optimizar la asignación de recursos para que los estudios se enfoquen en producir los animes que agradan más a las personas. 

In [2]:
CURRENT=os.getcwd()
ROOT=os.path.dirname(CURRENT)

config_f=load_config("config.yaml")


datos=pd.read_csv(os.path.join(ROOT,config_f["data"]["main"]))
datos = datos[datos['studio'] != 'add_some']
datos = datos[datos['demographics'] != 'not_available']
datos = datos[datos['genres'] != 'not_available']

In [3]:
data_preprocessed, segmentacion = preprocessing(datos)

### Número de grupos adecuado

![Número de grupos adecuado](images/optimal_clusters.png)

In [4]:
original, datos_procesados, cluster_assignments=clustering(data_preprocessed, segmentacion)

In [5]:
bright_green_scale = ["#66FF00", "#99FF33", "#CCFFCC"]
green_to_orange_scale = ["#66FF00", "#CCFF66",  "#FF9933"]
fig = px.box(original, x="Cluster", y="score", color="Cluster",
             category_orders={"Cluster": sorted(original['Cluster'].unique())},
             color_discrete_sequence=green_to_orange_scale)

fig.update_layout(
    title=dict(text="<b>Distribución de puntajes por clúster</b>", font=dict(color='white')),
    xaxis=dict(title="<b>Clúster</b>", titlefont=dict(color='white'), tickfont=dict(color='white'), linecolor='white', showgrid=False),
    yaxis=dict(title="<b>Puntaje</b>", titlefont=dict(color='white'), tickfont=dict(color='white'), linecolor='white', showgrid=False),
    autosize=False,
    width=800,
    height=500,
    plot_bgcolor='rgba(0,0,0,0)',  
    paper_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=50, r=50, b=100, t=100, pad=5)
)

fig.show()


In [6]:
bright_green_scale = ["#66FF00", "#CCFF99", "#CCFFCC"]
green_to_orange_scale = ["#66FF00", "#CCFF66",  "#FF9933"]

fig = px.histogram(original, x="Cluster", y="members", color="Cluster", histfunc='sum',
                   category_orders={"Cluster": sorted(original['Cluster'].unique())},
                   color_discrete_sequence=green_to_orange_scale)

fig.update_layout(
    title=dict(text="<b>Número de miembros por clúster</b>", font=dict(color='white')),
    xaxis=dict(title="<b>Clúster</b>", titlefont=dict(color='white'), tickfont=dict(color='white'), linecolor='white', showgrid=False),
    yaxis=dict(title="<b>Miembros</b>", titlefont=dict(color='white'), tickfont=dict(color='white'), linecolor='white', showgrid=False),
    autosize=False,
    width=800,
    height=500,
    plot_bgcolor='rgba(0,0,0,0)',  
    paper_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=50, r=50, b=100, t=100, pad=5)
)

fig.show()


In [7]:
# Inicializar la figura
fig = go.Figure()

# Añadir una traza para cada género
genres = ['action', 'comedy', 'ecchi', 'slice_of_life', 'sports', 'drama and romance', 'adventure and fantasy', 'mystery and horror', 'sci_fi and supernatural']
colors = ['lime', 'cyan', 'magenta', 'yellow', 'blue', 'green', 'orange', 'red', 'purple']  # Colores brillantes para cada género

for genre, color in zip(genres, colors):
    # Generar un histograma para cada género
    genre_counts = original[original[genre]==1]['Cluster'].value_counts().sort_index()

    # Añadir la traza a la figura
    fig.add_trace(go.Bar(x=genre_counts.index, y=genre_counts.values, name=genre, marker_color=color))

# Actualizar el diseño de la figura para que las barras estén apiladas
fig.update_layout(
    barmode='stack',
    title=dict(
        text="<b>Distribución de géneros por clúster</b>",
        font=dict(color='white'),
        pad=dict(t=100)  # Aumentamos el espacio en la parte superior
    ),
    xaxis=dict(title="<b>Clúster</b>", titlefont=dict(color='white'), tickfont=dict(color='white'), linecolor='white', showgrid=False),
    yaxis=dict(title="<b>Count</b>", titlefont=dict(color='white'), tickfont=dict(color='white'), linecolor='white', showgrid=False),
    autosize=False,
    width=800,
    height=600,
    plot_bgcolor='rgba(0,0,0,0)',  
    paper_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=50, r=50, b=100, t=150, pad=5),  # Aumentamos el margen superior
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1,
        xanchor="right",
        x=1,
        font=dict(color='white')  # Configuración del color de fuente del legend a blanco
    )
)

# Mostrar la figura
fig.show()



### Predicción del score de los animes

In [8]:
datos=pd.read_csv(os.path.join(ROOT,config_f["data"]["main"]))
genres=pd.read_csv(os.path.join(ROOT,config_f["data"]["cat_genres"]))
themes=pd.read_csv(os.path.join(ROOT,config_f["data"]["cat_themes"]))
datos = datos\
        .assign(
            themes = lambda df_: df_.themes\
                .str.replace("\[","", regex=True)\
                .str.replace("\]","", regex=True)\
                .str.replace("\'","", regex=True)\
                .str.replace(",'","", regex=True),
            genres = lambda df_: df_.genres\
                .str.replace("\[","", regex=True)\
                .str.replace("\]","", regex=True)\
                .str.replace("\'","", regex=True)\
                .str.replace(",'","", regex=True)
        )
# Calcula la frecuencia de cada estudio
studio_counts = datos['studio'].value_counts()
# Crea un conjunto de los estudios que aparecen 2 veces o menos
rare_studios = set(studio_counts[studio_counts <= 30].index)
datos['studio'] = datos['studio'].apply(lambda x: 'otros_estudios' if x in rare_studios else x)

In [9]:
numerical = ['number_episodes']
nominal = ['studio', 'genres', 'demographics', 'emission_type', 'year', 'month']
datos,numerical,nominal=impute_missing_drop_columns(datos,numerical,nominal)
y=datos["score"]
datos_xgboost=datos.drop(columns=["score","ranking","themes","title","url","first_emission","last_emission", 'members'])
genres_encoded = datos_xgboost['genres'].str.get_dummies(',')
datos_xgboost.drop(columns="genres",inplace=True)
    
categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])
    
preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, ['demographics', 'emission_type', 'year', 'month','studio']),
        ],
        remainder="passthrough")
    
preprocessor.fit(datos_xgboost)

# save the processor to disk
filename_process = "preprocesamiento_xgboost.pkl"
#Write the scaler to folder
with open(ROOT+"/"+config_f["modelos"]+filename_process, 'wb') as file_process:
    pickle.dump(preprocessor, file_process)

data_processed = preprocessor.transform(datos_xgboost)

data_processed= data_processed.toarray()

X_train, X_test, y_train, y_test = train_test_split(data_processed,y, test_size=0.3, random_state=42)

In [10]:
# Load best model object
best_model= pickle.load( open( ROOT+"/"+config_f["modelos"]+config_f['gs_best_model_name'], "rb" ) )

In [11]:
importances = best_model.feature_importances_

In [12]:
# Generar predicciones
y_pred = best_model.predict(X_test)


residuals = y_test - y_pred

fig = go.Figure()

# Agrega los residuos como puntos
fig.add_trace(go.Scatter(x=y_test, y=residuals,
                    mode='markers',
                    marker=dict(color='lime'),  # Cambiar el color de los marcadores
                    name='Residuales'))

# Agrega la línea de cero residuales
fig.add_shape(
        type="line",
        x0=y_test.min(),
        y0=0,
        x1=y_test.max(),
        y1=0,
        line=dict(
            color="white",   # Cambiar el color de la línea
            width=3,
            dash="dashdot", # Cambiar el estilo de la línea a punteado
        )
)

# Actualizar el diseño de la figura
fig.update_layout(
    title=dict(text="<b>Gráfica de residuales</b>", font=dict(color='white')), 
    xaxis=dict(title="<b>Verdaderos scores</b>", titlefont=dict(color='white'), tickfont=dict(color='white'), linecolor='white', showgrid=False),
    yaxis=dict(title="<b>Residuales</b>", titlefont=dict(color='white'), tickfont=dict(color='white'), linecolor='white', showgrid=False),
    autosize=False,
    width=800,
    height=500,
    plot_bgcolor='rgba(0,0,0,0)',  # Hacer que el fondo de la gráfica sea transparente
    paper_bgcolor='rgba(0,0,0,0)', # Hacer que el fondo del papel sea transparente
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=5
    )
)
fig.show()

In [13]:
# Obtén los nombres de las características de la transformación de datos preprocesados
ohe_feature_names = list(preprocessor.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out())

# Nominal features after one hot encoding
nominal_encoded = ohe_feature_names

# Features that were not encoded
not_encoded = numerical

# Combine the two lists together to get the final feature names
feature_names = nominal_encoded + not_encoded
# Eliminar espacios en blanco al inicio de las características
feature_names = [name.lstrip() for name in feature_names]
# Eliminar duplicados
feature_names = list(dict.fromkeys(feature_names))
# Crea un DataFrame con las importancias y los nombres de las características
feature_importances = pd.DataFrame({"feature": feature_names, "importance": importances})

# Ordena el DataFrame en orden descendente de importancia
feature_importances = feature_importances.sort_values("importance", ascending=False)

# Prepara los datos para el gráfico de barras
top_features = feature_importances["feature"][:15]
top_importances = feature_importances["importance"][:15]

In [14]:
# Crea un DataFrame con las importancias y los nombres de las características
feature_importances = pd.DataFrame({"feature": feature_names, "importance": importances})

# Ordena el DataFrame en orden descendente de importancia
feature_importances = feature_importances.sort_values("importance", ascending=False)

# Crea la figura y los datos de las barras
fig = go.Figure(data=[go.Bar(
    x=top_importances,
    y=top_features,
    orientation='h',
    marker_color='lime'   # Cambiar el color de las barras
)])

# Establece el estilo del gráfico
fig.update_layout(
    title=dict(text="<b>Top 10 Características Importantes para la regresión</b>", font=dict(color='white')), 
    xaxis=dict(title="<b>Importancia de la Característica</b>", titlefont=dict(color='white'), tickfont=dict(color='white'), linecolor='white', showgrid=False),
    yaxis=dict(title="<b>Característica</b>", titlefont=dict(color='white'), tickfont=dict(color='white'), linecolor='white', showgrid=False, autorange="reversed"),
    autosize=False,
    width=800,
    height=500,
    plot_bgcolor='rgba(0,0,0,0)',  # Hacer que el fondo de la gráfica sea transparente
    paper_bgcolor='rgba(0,0,0,0)', # Hacer que el fondo del papel sea transparente
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=5
    )
)

# Muestra el gráfico
fig.show()
